In [27]:
# Warning control
import warnings
warnings.filterwarnings('ignore')


# Import Groq
import os
import yaml
from crewai import Agent, Task, Crew, LLM

In [28]:
llm = LLM(
    model="groq/llama3-8b-8192",
    api_key=os.getenv("GROQ_API_KEY")
)

In [29]:
# Define file paths for YAML configurations
files = {
    'agents': 'config/agents.yaml',
    'tasks': 'config/tasks.yaml'
}

In [30]:
# Load configurations from YAML files
configs = {}
for config_type, file_path in files.items():
    with open(file_path, 'r') as file:
        configs[config_type] = yaml.safe_load(file)
agents_config = configs['agents']
tasks_config = configs['tasks']

In [31]:
from typing import List
from pydantic import BaseModel, Field

class TaskEstimate(BaseModel):
    task_name: str = Field(..., description="Name of the task")
    estimated_time_hours: float = Field(..., description="Estimated time to complete the task in hours")
    required_resources: List[str] = Field(..., description="List of resources required to complete the task")

class Milestone(BaseModel):
    milestone_name: str = Field(..., description="Name of the milestone")
    tasks: List[str] = Field(..., description="List of task IDs associated with this milestone")

class ProjectPlan(BaseModel):
    tasks: List[TaskEstimate] = Field(..., description="List of tasks with their estimates")
    milestones: List[Milestone] = Field(..., description="List of project milestones")

In [32]:
# Creating Agents
project_planning_agent = Agent(
  config=agents_config['project_planning_agent'],
  llm=llm
)

estimation_agent = Agent(
  config=agents_config['estimation_agent'],
  llm=llm
)

resource_allocation_agent = Agent(
  config=agents_config['resource_allocation_agent'],
  llm=llm
)

In [33]:

# Creating Tasks
task_breakdown = Task(
  config=tasks_config['task_breakdown'],
  agent=project_planning_agent
)

time_resource_estimation = Task(
  config=tasks_config['time_resource_estimation'],
  agent=estimation_agent
)

resource_allocation = Task(
  config=tasks_config['resource_allocation'],
  agent=resource_allocation_agent,
  output_pydantic=ProjectPlan # This is the structured output we want
)

In [34]:

# Creating Crew
crew = Crew(
  agents=[
    project_planning_agent,
    estimation_agent,
    resource_allocation_agent
  ],
  tasks=[
    task_breakdown,
    time_resource_estimation,
    resource_allocation
  ],
  verbose=True
)

Overriding of current TracerProvider is not allowed


In [35]:
from IPython.display import display, Markdown

project = 'Website'
industry = 'Technology'
project_objectives = 'Create a website for a small business'
team_members = """
- John Doe (Project Manager)
- Jane Doe (Software Engineer)
- Bob Smith (Software Engineer)
- Alice Johnson (Designer)
- Tom Brown (QA Engineer)
"""
# building a landing page using the following requirements

project_requirements = """
- Create a responsive design that works well on desktop and mobile devices
- Implement a modern, visually appealing user interface with a clean look
- Develop a user-friendly navigation system with intuitive menu structure
- Include an "About Us" page highlighting the company's history and values
- Design a "Services" page showcasing the business's offerings with descriptions
- Create a "Contact Us" page with a form and integrated map for communication
- Implement a blog section for sharing industry news and company updates
- Ensure fast loading times and optimize for search engines (SEO)
- Integrate social media links and sharing capabilities
- Include a testimonials section to showcase customer feedback and build trust
"""

# Format the dictionary as Markdown for a better display in Jupyter Lab
formatted_output = f"""
**Project Type:** {project}

**Project Objectives:** {project_objectives}

**Industry:** {industry}

**Team Members:**
{team_members}
**Project Requirements:**
{project_requirements}
"""
# Display the formatted output as Markdown
display(Markdown(formatted_output))


**Project Type:** Website

**Project Objectives:** Create a website for a small business

**Industry:** Technology

**Team Members:**

- John Doe (Project Manager)
- Jane Doe (Software Engineer)
- Bob Smith (Software Engineer)
- Alice Johnson (Designer)
- Tom Brown (QA Engineer)

**Project Requirements:**

- Create a responsive design that works well on desktop and mobile devices
- Implement a modern, visually appealing user interface with a clean look
- Develop a user-friendly navigation system with intuitive menu structure
- Include an "About Us" page highlighting the company's history and values
- Design a "Services" page showcasing the business's offerings with descriptions
- Create a "Contact Us" page with a form and integrated map for communication
- Implement a blog section for sharing industry news and company updates
- Ensure fast loading times and optimize for search engines (SEO)
- Integrate social media links and sharing capabilities
- Include a testimonials section to showcase customer feedback and build trust



In [36]:
# The given Python dictionary
inputs = {
  'project_type': project,
  'project_objectives': project_objectives,
  'industry': industry,
  'team_members': team_members,
  'project_requirements': project_requirements
}


In [37]:

result = crew.kickoff(
  inputs=inputs
)

# Agent: The Ultimate Project Planner
## Task: Carefully analyze the project_requirements for the Website project and break them down into individual tasks. Define each task's scope in detail, set achievable timelines, and ensure that all dependencies are accounted for:

- Create a responsive design that works well on desktop and mobile devices
- Implement a modern, visually appealing user interface with a clean look
- Develop a user-friendly navigation system with intuitive menu structure
- Include an "About Us" page highlighting the company's history and values
- Design a "Services" page showcasing the business's offerings with descriptions
- Create a "Contact Us" page with a form and integrated map for communication
- Implement a blog section for sharing industry news and company updates
- Ensure fast loading times and optimize for search engines (SEO)
- Integrate social media links and sharing capabilities
- Include a testimonials section to showcase customer feedback and build tru

In [38]:
result.pydantic.dict()

{'tasks': [{'task_name': 'Define Project Requirements and Scope',
   'estimated_time_hours': 3.0,
   'required_resources': ['John Doe (Project Manager)']},
  {'task_name': 'Design Initial Page Structure',
   'estimated_time_hours': 2.0,
   'required_resources': ['Alice Johnson (Designer)']},
  {'task_name': 'Design Responsive Web Pages',
   'estimated_time_hours': 2.0,
   'required_resources': ['Alice Johnson (Designer)']},
  {'task_name': 'Develop Responsive UI',
   'estimated_time_hours': 4.0,
   'required_resources': ['Jane Doe (Software Engineer), Bob Smith (Software Engineer)']},
  {'task_name': 'Implement Navigation System',
   'estimated_time_hours': 2.0,
   'required_resources': ['Jane Doe (Software Engineer), Tom Brown (QA Engineer)']},
  {'task_name': 'Develop Blog Section',
   'estimated_time_hours': 4.0,
   'required_resources': ['Bob Smith (Software Engineer), Tom Brown (QA Engineer)']},
  {'task_name': 'Implement SEO Optimization',
   'estimated_time_hours': 2.0,
   'requ

In [40]:
import pandas as pd
tasks = result.pydantic.dict()['tasks']
df_tasks = pd.DataFrame(tasks)

# Display the DataFrame as an HTML table
df_tasks.style.set_table_attributes('border="1"').set_caption("Task Details").set_table_styles(
    [{'selector': 'th, td', 'props': [('font-size', '120%')]}]
)

,task_name,estimated_time_hours,required_resources
0,Define Project Requirements and Scope,3.000000,['John Doe (Project Manager)']
1,Design Initial Page Structure,2.000000,['Alice Johnson (Designer)']
2,Design Responsive Web Pages,2.000000,['Alice Johnson (Designer)']
3,Develop Responsive UI,4.000000,"['Jane Doe (Software Engineer), Bob Smith (Software Engineer)']"
4,Implement Navigation System,2.000000,"['Jane Doe (Software Engineer), Tom Brown (QA Engineer)']"
5,Develop Blog Section,4.000000,"['Bob Smith (Software Engineer), Tom Brown (QA Engineer)']"
6,Implement SEO Optimization,2.000000,"['John Doe (Project Manager), Jane Doe (Software Engineer)']"
7,Integrate Social Media Links,2.000000,"['Bob Smith (Software Engineer), Tom Brown (QA Engineer)']"
8,Create Testimonials Section,2.000000,"['Alice Johnson (Designer), Jane Doe (Software Engineer)']"
9,Quality Assurance and Testing,5.000000,"['Tom Brown (QA Engineer), Jane Doe (Software Engineer), Bob Smith (Software Engineer)']"
